In [236]:
# import needed library
from sklearn.datasets import load_iris
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import math
import numpy as np

# train test split 
from sklearn.model_selection import train_test_split

# Kfold
from sklearn.model_selection import KFold

# regex
import re
 

In [237]:
#load iris dataset
data_iris = load_iris()
iris_X, iris_y = load_iris(return_X_y=True)
feature_iris = data_iris['feature_names']

In [238]:
#transform iris into dataframe
iris_X=pd.DataFrame(iris_X)
iris_y=pd.DataFrame(iris_y)

In [239]:
#create index so be merge
iris_X=iris_X.reset_index()
iris_y=iris_y.reset_index()

In [240]:
iris_X.rename(columns = {0:feature_iris[0],1:feature_iris[1],2:feature_iris[2],3:feature_iris[3]}, inplace = True)


In [241]:
iris_y.rename(columns = {0:'target'}, inplace = True) 

In [242]:
#merge dataset iris
iris=iris_X.merge(iris_y)

In [243]:
#drop index
iris.drop("index",axis=1,inplace=True)

In [244]:
def entropy(parsed_data, target_attribute):
    parsed_value_target = {}
    total_value_target = 0
  
    for i in parsed_data[target_attribute]:
        if i is not None:
            if i not in parsed_value_target:
                parsed_value_target[i] = 1
            else:
                parsed_value_target[i] += 1

            total_value_target += 1
  
    log_result = 0

    for i in parsed_value_target:
        log_result += float(parsed_value_target[i])/total_value_target * math.log((float(parsed_value_target[i])/total_value_target), 2)
  
    return -1 * log_result

In [245]:
# hasn't handle after universal entropy
def information_gain(data, gain_attribute, target_attribute):
    gain_result = 0
    attribute_entropy_result = 0
    parsed_attribute_count = {}
    total_attribute_count = 0
    
    for i in data[gain_attribute]:
        if i is not None:
            if i not in parsed_attribute_count:
                parsed_attribute_count[i] = 1
            else:
                parsed_attribute_count[i] += 1
            
            total_attribute_count += 1
    
    for i in parsed_attribute_count:
        parsed_data = data.loc[data[gain_attribute]==i]
        attribute_entropy_result += float(parsed_attribute_count[i])/total_attribute_count * entropy(parsed_data, target_attribute)    

    gain_result += entropy(data,target_attribute) + (-1 * attribute_entropy_result)
    return gain_result

In [246]:
def split_information(data, gain_attribute):
    res = entropy(data, gain_attribute)
    if(res==0):
        res=0.00000001
    return res

In [247]:
def gain_ratio(data, gain_attribute, target_attribute):
    res = information_gain(data, gain_attribute, target_attribute) / split_information(data, gain_attribute)
    return res

In [248]:
def gain_ratio_continous(data, gain_attribute, target_attribute):
    res = information_gain_continous(data, gain_attribute, target_attribute)[0] / split_information(data, gain_attribute)
    return res,information_gain_continous(data, gain_attribute, target_attribute)[1]

In [249]:
def best_attribute(data,target_attribute,is_IG):
    gain_attribute = {
        'value': 0,
        'name': ''
    }
    
    
    for i in data.columns:
        if (i != target_attribute):
            if is_IG:
                if information_gain(data, i, target_attribute) > gain_attribute['value']:
                    gain_attribute['value'] = information_gain(data, i, target_attribute)
                    gain_attribute['name'] = i
            else:
                if gain_ratio(data, i, target_attribute) > gain_attribute['value']:
                    gain_attribute['value'] = gain_ratio(data, i, target_attribute)
                    gain_attribute['name'] = i
                

    return gain_attribute['name']

In [250]:
import math

class Node:
    def __init__(self, attribute=None, label=None, vertex=None):
        self.attribute = attribute
        self.label = label
        self.vertex = vertex
        self.children = {}
        self.most_common_label = None
    
    def set_most_common_label(self, most_common_label):
        self.most_common_label = most_common_label
        
    def get_most_common_label(self):
        return self.most_common_label
        
    def setAttribute(self, attribute):
        self.attribute = attribute

    def setLabel(self, label):
        self.label = label
        
    def setVertex(self, vertex):
        self.vertex = vertex
  
    def addChildren(self, attributeValue, node):
        self.children[attributeValue] = node
    
    def getChildren(self):
        return self.children
    
    def getLabel(self):
        return self.label
    
    def getVertex(self):
        return self.vertex

In [251]:
def get_most_common_label(data, target_attribute):
    parsed_value_target = {}
  
    for i in data[target_attribute]:
        if i is not None:
            if i not in parsed_value_target:
                parsed_value_target[i] = 1
            else:
                parsed_value_target[i] += 1

    most_common = {
        'value': 0,
        'name': ''
    }
    
    for i in parsed_value_target:
        if parsed_value_target[i] > most_common['value']:
            most_common['value'] = parsed_value_target[i]
            most_common['name'] = i
    
    return most_common['name']

In [252]:
def most_common_label_target(data,target_attribute):
    most_comm = None
    occ = 0;
    for i in data[target_attribute].unique():
        if data[data[target_attribute] == i].shape[0] >  occ:
            most_comm = i
            occ = data[data[target_attribute] == i].shape[0]
    return most_comm

In [253]:
def id3(data, target_attribute, is_IG):
    node = Node()
    if data[target_attribute].nunique()==1:
        node.setLabel(data[target_attribute].unique()[0])
        return node
    
    elif len(data.columns)==1:
        node.setLabel(get_most_common_label(data, target_attribute))
        return node
    
    else:
        best_attribute_ = best_attribute(data,target_attribute,is_IG)
        node.setAttribute(best_attribute_)
        for i in data[best_attribute_].unique():
            node.addChildren(i,id3(data.loc[data[best_attribute_]==i],target_attribute,is_IG))
            node.set_most_common_label(most_common_label_target(data,target_attribute))        
            
    return node

In [254]:
def print_tree(node,depth):
    if node.label is not None: 
        print("    "*(depth+1) +str(node.label))
    else:
        print("    "*depth + "["+ node.attribute +"]")
        for i in node.children:
            print("----"*(depth+1) +str(i))
            print_tree(node.children[i],depth+1)        

In [255]:
def copy_tree(node):
    temp_node = Node()
    if node.label is not None: 
        temp_node.setLabel(node.label)
    else:
        temp_node.setAttribute(node.attribute)
        for i in node.children:
            temp= Node()
            temp= node.children[i]
            temp_node.addChildren(i, temp)
    return temp_node

In [256]:
def check_tree(node,data,index,result, target_attribute):
    if node.label is not None: 
        result.append(node.getLabel())
    else:
        if data.loc[index, node.attribute] is None:
            result.append(node.get_most_common_label())
        for i in node.children:
            if i==data.loc[index,node.attribute]:
                check_tree(node.children[i],data,index,result, target_attribute)

In [257]:
def pred(data,model,target_attribute):
    result = []
    for i in range(len(data)):
        check_tree(model,data[i:i+1],i,result, target_attribute)
    data = {target_attribute:result}
    return pd.DataFrame(data)
    

In [258]:
# hasn't handle after universal entropy
def information_gain_continous(data, gain_attribute, target_attribute):
    gain_result = 0
    save_boundary = -1
    min_entropy = 999999 
    data = data.sort_values(gain_attribute)
    data=data.reset_index().drop('index',axis=1)
    length_data = len(data)
    for i in range(length_data):
        if (i!=length_data-1):
            if (data.loc[i,target_attribute] != data.loc[i+1,target_attribute]):
                temp_boundary = (data.loc[i,gain_attribute] + data.loc[i+1,gain_attribute])/2
                parsed_data_upper = data.loc[data[gain_attribute]>=temp_boundary]
                parsed_data_lower = data.loc[data[gain_attribute]<temp_boundary]
                len_parsed = len(parsed_data_upper)               
                temp_entropy = float(len_parsed)/length_data * entropy(parsed_data_upper, target_attribute) + float((length_data-len_parsed))/length_data * entropy(parsed_data_lower, target_attribute)     
                if temp_entropy < min_entropy:
                    #print(gain_attribute, temp_boundary, temp_entropy)
                    min_entropy = temp_entropy 
                    save_boundary = temp_boundary

    gain_result += entropy(data,target_attribute) + (-1 * min_entropy)
    return gain_result,save_boundary

In [259]:
information_gain_continous(iris, 'petal length (cm)', 'target')

(0.9182958340544894, 2.45)

In [260]:
def best_attribute_c45(data,target_attribute,is_IG):
    gain_attribute = {
        'value': 0,
        'name': '',
        'boundary': -99999999
    }
    
    
    for i in data.columns:
        if (i != target_attribute):
            if is_IG:
                if data[i].dtypes in [pd.np.dtype('float64'), pd.np.dtype('float32')]:
                    ig = information_gain_continous(data, i, target_attribute) 
                    if ig[0] > gain_attribute['value']:
                            gain_attribute['value'] = ig[0]
                            gain_attribute['name'] = i
                            gain_attribute['boundary'] = ig[1]
                else:
                    if information_gain(data, i, target_attribute) > gain_attribute['value']:
                            gain_attribute['value'] = information_gain(data, i, target_attribute)
                            gain_attribute['name'] = i
            else:
                if data[i].dtypes in [pd.np.dtype('float64'), pd.np.dtype('float32')]:
                    ig = gain_ratio_continous(data, i, target_attribute) 
                    if ig[0] > gain_attribute['value']:
                            gain_attribute['value'] = ig[0]
                            gain_attribute['name'] = i
                            gain_attribute['boundary'] = ig[1]
                else:
                    if gain_ratio(data, i, target_attribute) > gain_attribute['value']:
                            gain_attribute['value'] = gain_ratio(data, i, target_attribute)
                            gain_attribute['name'] = i

    return gain_attribute['name'],round(gain_attribute['boundary'],2)

In [261]:
best_attribute_c45(iris,'target',False)

('petal width (cm)', 0.8)

In [262]:
def c45(data, target_attribute,is_IG):
    node = Node()
    if data[target_attribute].nunique()==1:
        node.setLabel(data[target_attribute].unique()[0])
        return node
    
    elif len(data.columns)==1:
        node.setLabel(get_most_common_label(data, target_attribute))
        return node
    
    else:
        best_attribute_,bound  = best_attribute_c45(data,target_attribute,is_IG)
        #print(best_attribute_)
        node.setAttribute(best_attribute_)
        node.set_most_common_label(most_common_label_target(data,target_attribute))
        if bound==-99999999:
            for i in data[best_attribute_].unique():
                node.addChildren(i,c45(data.loc[data[best_attribute_]==i],target_attribute,is_IG))
        else:
            node.addChildren('>='+str(bound),c45(data.loc[data[best_attribute_]>=bound],target_attribute,is_IG))
            node.addChildren('<'+str(bound),c45(data.loc[data[best_attribute_]<bound],target_attribute,is_IG))
    return node

In [263]:
def c45_prun(data, target_attribute,is_IG,vertex_=None):
    node = Node(vertex = vertex_)
    if data[target_attribute].nunique()==1:
        node.setLabel(data[target_attribute].unique()[0])
        return node
     
    elif len(data.columns)==1:
        node.setLabel(get_most_common_label(data, target_attribute))
        return node
    
    else:
        best_attribute_,bound  = best_attribute_c45(data,target_attribute,is_IG)
        #print(best_attribute_)
        node.setAttribute(best_attribute_)
        node.set_most_common_label(most_common_label_target(data,target_attribute))
        if bound==-99999999:
            for i in data[best_attribute_].unique():
                node.addChildren(i,c45_prun(data.loc[data[best_attribute_]==i],target_attribute,is_IG,i))
        else:
            node.addChildren('>='+str(bound),c45_prun(data.loc[data[best_attribute_]>=bound],target_attribute,is_IG,'>='+str(bound)))
            node.addChildren('<'+str(bound),c45_prun(data.loc[data[best_attribute_]<bound],target_attribute,is_IG,'<'+str(bound)))
    return node

In [264]:
c45(iris, "target",True)

In [265]:
print_tree(c45(iris, "target",True),0)

[petal length (cm)]
---->=2.45
    [petal width (cm)]
-------->=1.75
        [sepal width (cm)]
------------>=3.15
            [sepal length (cm)]
---------------->=6.05
                    2
----------------<6.05
                    1
------------<3.15
                2
--------<1.75
        [petal length (cm)]
------------>=4.95
            [petal width (cm)]
---------------->=1.55
                [sepal length (cm)]
-------------------->=6.95
                        2
--------------------<6.95
                        1
----------------<1.55
                    2
------------<4.95
            [petal width (cm)]
---------------->=1.65
                    2
----------------<1.65
                    1
----<2.45
        0


In [266]:
def check_tree_c45(node,data,index,result, target_attribute):
    if node.label is not None: 
        result.append(node.getLabel())
    else:
        if data.loc[index, node.attribute] is None:
            result.append(node.get_most_common_label())
        else:
            for i in node.children:
                #print(i)
                if i[0]=='<':
                    #print(i,2)
                    bound=float(i[1:])
                    if bound>data.loc[index,node.attribute]:
                        #print(data.loc[index,node.attribute])
                        check_tree_c45(node.children[i],data,index,result, target_attribute)
                elif i[0]=='>':
                    bound=float(i[2:])
                    #print(i,1)
                    if bound<=data.loc[index,node.attribute]:
                        #print(data.loc[index,node.attribute])
                        check_tree_c45(node.children[i],data,index,result, target_attribute)
                else:
                    if i==data.loc[index,node.attribute]:
                        check_tree(node.children[i],data,index,result, target_attribute)
                    

In [267]:
def pred_c45(data,model,target_attribute):
    result = []
    for i in range(len(data)):
        check_tree_c45(model,data[i:i+1],i,result, target_attribute)
    data = { target_attribute: data[target_attribute]
            ,'prediction':result}
    return pd.DataFrame(data)
    

In [268]:
p=iris

In [269]:
p=p.drop('target',axis=1)

In [270]:
p=p.reset_index().drop('index',axis=1)

In [271]:
p['sepal width (cm)'] = None

In [272]:
def accuracy(pred,data):
    cnt = 0
    for i in range(len(pred)):
        if pred.loc[i] == data.loc[i]:
            cnt+=1
    return cnt*100/len(pred)

In [273]:
def get_data_validate(data):
    data_column = data.columns
    data_10 = pd.DataFrame(columns=data_column)
    data_90 = pd.DataFrame(columns=data_column)
    #print(data_column)
    count_10 = 0
    count_90 = 0
    
    for i in range(data.shape[0]):
        if(i%10 == 1):
            # Pass the row elements as key value pairs to append() function 
            data_10 = data_10.append(data.loc[[i]] , ignore_index=True)
        else:
            data_90 = data_90.append(data.loc[[i]] , ignore_index=True)
    return data_10, data_90

In [274]:
def prune_tree(node, attribute, vertex):
    if(node.label is not None):
        return node,0
    elif(node.attribute == attribute and node.vertex == vertex):
        node.children = {}
        node.attribute = None
        node.label = node.most_common_label
        
        return node,1
    else:
        for i in node.children:
            a,b = prune_tree(node.children[i], attribute, vertex)
            
        return node,b

In [275]:
def post_pruning(data, node, current_node, children_node, target_attribute):
    if children_node == 0:
        return node
    else:
        for i in current_node.children:
            if current_node.vertex is None: 
                #print(i,1)
                post_pruning(data, node, current_node.children[i], children_node-1, target_attribute)
            else:
                if current_node.attribute is not None:
                    data10, data90 = get_data_validate(data)
                    #print(current_node.attribute,2)
                    save = copy_tree(current_node)
                    temp_node,c = prune_tree(node, current_node.attribute, current_node.vertex)
                    train=data90.drop(target_attribute,axis=1)
                    #print_tree(node,0)
                    temp_node_pred = pred_c45(train, node, target_attribute)
                    #print(temp_node_pred)
                    temp_node_accuracy = accuracy(temp_node_pred[target_attribute], data90[target_attribute])
                    #print(temp_node_accuracy)
                    if temp_node_accuracy == 100:
                        post_pruning(data, temp_node, current_node, children_node, target_attribute)
                    else :
                        
                        node.addChildren(current_node.vertex, save)
                        #print_tree(node,0)


                        post_pruning(data, node, save.children[i], children_node, target_attribute)
        return node

### ANN

In [276]:
def sigmoid(x):
    return 1/(1+math.exp(-x))

In [277]:
def sign(x):
    if (x>0.5):
        return 1
    else:
        return 0

In [278]:
def count_error(target, output):
    return 0.5*(target-output)*(target-output)

In [279]:
class Neuron:
    def __init__(self, out=None, w=None, is_used=None, error=None):
        self.out = out
        self.w = []
        self.is_used = is_used
        self.error = 0
        self.deltaW = []
        
    def set_out(self, out):
        self.out = out
        
    def get_out(self):
        return self.out
    
    def add_deltaW(self, value):
        self.deltaW.append(value)
        
    def get_deltaW(self, index):
        return self.deltaW[index]
    
    def get_arrdW(self):
        return self.deltaW
    
    def set_deltaW(self, index, value):
        self.deltaW[index] = value
        
    def add_w(self, value):
        self.w.append(value)
        
    def set_w(self, index, value):
        self.w[index] = value
    
    def get_w(self, index):
        return self.w[index]
    
    def get_arrW(self):
        return self.w
    
    def set_is_used(self, is_used):
        self.is_used = is_used
        
    def get_is_used(self):
        return self.is_used
    
    def set_error(self, error):
        self.error = error
    
    def get_error(self):
        return self.error

In [280]:
x = [Neuron() for i in range (5)]
y = [x for i in range(5)]

In [281]:
def LenCol(arr):
    return len(arr)
def LenRow(inp,hid,out):
    return max(inp,max(hid)+1,out)
def MakeMatrix(row,col):
    return [[Neuron() for i in range (col)] for j in range(row)]

In [282]:
def printMatrixMLP(matrix):
    for i in range(len(matrix)):
        temp_matrix_out = []
        for j in range(len(matrix[i])):
            temp_matrix_out.append(matrix[i][j].get_arrW())
        print(temp_matrix_out)

In [283]:
def convTarget(output,unique):
    res = []
    for i in range(unique):
        if (i!=output):
            res.append(0)
        else:
            res.append(1)
    return res
    

In [284]:
def resetDeltaWeight(mat,layer):
    for i in range(len(layer)-1):
        for j in range(layer[i]):
            for k in range(layer[i+1]):
                mat[j][i].set_deltaW(k,0)

In [285]:
def initW(mat,layer):
    for i in range(len(layer)-1):
        for j in range(layer[i]):
            for k in range(layer[i+1]):
                mat[j][i].add_w(np.random.uniform(-1,1)) 
                mat[j][i].add_deltaW(0)

In [286]:
def feedforward(mat,layer):
    for i in range(1,len(layer)):
        if (i!=len(layer)-1):
            mat[layer[i]][i].set_out(1)
        for j in range(layer[i]):
            net = 0;
            for k in range(layer[i-1]):
                #print(k,i-1,j)
                net = net + mat[k][i-1].get_out()*mat[k][i-1].get_w(j)
#             if (i!=len(layer)-1):
            mat[j][i].set_out(sigmoid(net))
#             else:
#                 mat[j][i].set_out(sign(sigmoid(net)))

In [287]:
def get_error_total(matrix, n_output, target_arr):
    output_col = len(matrix[0])-1
    total_error = 0
    for i in range(n_output):
        total_error += 0.5*(target_arr[i] - matrix[i][output_col].get_out())**2
    
    return total_error

In [288]:
def set_output_error(matrix, row, col, target_value):
    output_value = matrix[row][col].get_out()
    #print(output_value, target_value)
#     print(output_value * (1 - output_value) * (target_value - output_value))
    matrix[row][col].set_error(output_value * (1 - output_value) * (target_value - output_value))

In [289]:
def set_hidden_error(matrix, row, col):
    output_value = matrix[row][col].get_out()
    delta_weight_error = 0
    for i in range(len(matrix[row][col].w)):
        delta_weight_error += matrix[row][col].get_w(i) * matrix[i][col+1].get_error()
    matrix[row][col].set_error(output_value * (1 - output_value) * delta_weight_error)

In [290]:
def update_delta_weight(matrix, row, col, idx , learning_rate):
    #print(matrix[i][col+1].get_error(),matrix[row][col].get_out(),row,col)
    #print(row,col+1)
    delta_weight = learning_rate * matrix[row][col].get_error() * matrix[idx][col-1].get_out()
    #print(delta_weight)
    
    current_weight = matrix[idx][col-1].get_deltaW(row)
    matrix[idx][col-1].set_deltaW(row, current_weight + delta_weight)
    #print(idx,col-1,row,current_weight + delta_weight)

In [291]:
def update_weight(mat,layer):
    for i in range(len(layer)-1):
        for j in range(layer[i]):
            for k in range(layer[i+1]):
                curr_weight = mat[j][i].get_w(k)
                delta_weight = mat[j][i].get_deltaW(k)
                mat[j][i].set_w(k,curr_weight + delta_weight) 

In [292]:
def backpropagation(matrix, layer, learning_rate, target_arr):
    for i in range(len(layer)-1, 0, -1):
        if i == len(layer)-1:
            for j in range(layer[i]):
                set_output_error(matrix, j, i, target_arr[j])
        else:
            for j in range(layer[i]):
                set_hidden_error(matrix, j, i)
        for j in range(layer[i-1]):
            for k in range(layer[i]):
                update_delta_weight(matrix, k, i, j, learning_rate)

In [293]:
hidden_node = [3,4]
def MLP(data,target, hidden_node, epochs, learning_rate):
    output_node = data[target].nunique()
    input_node = len(data.columns)
    hidden_node.insert(0,input_node)
    hidden_node.append(output_node)
    layer_node = hidden_node
    mlp = MakeMatrix(LenRow(input_node,hidden_node,output_node),LenCol(hidden_node))
    initW(mlp,layer_node)
    train = data.drop(target,axis=1)
    batch = len(data)/10;
    for k in range(epochs):
        error = 0
        for i in range(int(batch)):
            for j in range(10):
                for index,col in enumerate(train.columns):
                    mlp[index][0].set_out(data[col][j+i*10])
                mlp[input_node-1][0].set_out(1)
                feedforward(mlp,layer_node)
                backpropagation(mlp, layer_node, learning_rate, convTarget(data[target][j+i*10],output_node))
                error += get_error_total(mlp, len(convTarget(data[target][j+i*10],output_node)), convTarget(data[target][j+i*10],output_node))
            update_weight(mlp,layer_node)
            resetDeltaWeight(mlp,layer_node)
        print(error)
    print()
    print()
    printMatrixMLP(mlp)

    return mlp,layer_node

In [294]:
def predMyMLP(data, target, model, output_layer):
    out = []
    mlp = model[0]
    for i in range(0,data.shape[0],1):
        for index,col in enumerate(data.columns):
            mlp[index][0].set_out(data[col][i])
        mlp[len(data.columns)-1][0].set_out(1)
        feedforward(mlp,model[1])
        output_col = len(mlp[0])-1
        #print(output_col)
        mx=0
        res=-1
        for j in range(output_layer):
            # print(mlp[j][output_col].get_out())
            if mlp[j][output_col].get_out()>mx:
                mx = mlp[j][output_col].get_out()
                res = j
        # print()
        out.append(res)
    data = { target: data[target],
             'prediction':out}
    return pd.DataFrame(data)


# Testing

In [295]:
class_list = iris['target'].unique()

# Create confusion matrix from testing result of a model
#
# Input:
#    pandas.DataFrame   prediction_data
#    String             actual_column
#    String             prediction_column
#    Array              class_list
# Output:
#    pandas.DataFrame   confusion_matrix
#
def confusion_matrix(prediction_data, actual_column, prediction_column, class_list):
    actual_data = pd.Categorical(prediction_data[actual_column], categories=class_list)
    prediction_data = pd.Categorical(prediction_data[prediction_column], categories=class_list)
    confusion_matrix = pd.crosstab(actual_data, prediction_data, rownames=['Actual'], colnames=['Predicted'], dropna=False)
    return confusion_matrix

In [296]:
# Get accuracy from testing result of a model
#
# Input:
#    pandas.DataFrame   prediction_data
#    String             actual_column
#    String             prediction_column
# Output:
#    float              accuracy
#
def testing_accuracy(prediction_data, actual_column, prediction_column):
    count = 0
    for index, row in prediction_data.iterrows():
        if(row[actual_column] == row[prediction_column]):
            count = count + 1
    return round(count/prediction_data.shape[0],2)
            


## Train Test Split

In [297]:
iris_X.drop('index',axis=1,inplace=True);
iris_y.drop('index',axis=1,inplace=True);

In [298]:
# Testing DTL
X_train, X_test, y_train, y_test = train_test_split(iris_X, iris_y, test_size=0.1, random_state=42)

In [299]:
def mergeData(X_train, X_test, y_train, y_test):
    X_train = X_train.reset_index()
    y_train = y_train.reset_index()
    X_test = X_test.reset_index()
    y_test = y_test.reset_index()
    trainData = X_train.merge(y_train)
    testData = X_test.merge(y_test)
    trainData.drop("index",axis=1,inplace=True)
    testData.drop("index",axis=1,inplace=True)
    return trainData,testData

In [300]:
trainData,testData = mergeData(X_train, X_test, y_train, y_test)

In [301]:
model = c45(trainData, "target",True)
prediction_dtl = pred_c45(testData, model, "target")
print(prediction_dtl)

    target  prediction
0        1           1
1        0           0
2        2           2
3        1           1
4        1           1
5        0           0
6        1           1
7        2           2
8        1           1
9        1           1
10       2           2
11       0           0
12       0           0
13       0           0
14       0           0


In [302]:
def create_file_DTL(node, depth):
    

    if node.label is not None: 
        file.write(" "*(depth+1) +str(node.label))
        file.write('\n')
    else:
        file.write(" "*depth + "["+ node.attribute +"]")
        file.write('\n')
        for i in node.children:
            file.write("----"*(depth+1) +str(i))
            create_file_DTL(node.children[i],depth+1)        
    
# file = open("newfile.txt", "r")
# print(file.read())
# file.close()

In [303]:
file = open("text_dtl.txt", "w")
create_file_DTL(model, 0)
file.close()

In [304]:
print(confusion_matrix(prediction_dtl,'target','prediction', class_list))

Predicted  0  1  2
Actual            
0          6  0  0
1          0  6  0
2          0  0  3


In [305]:
testing_accuracy(prediction_dtl,'target','prediction')

1.0

In [306]:
hidden_node = [4,4,2]
x = MLP(trainData, 'target', hidden_node,1500,0.1)

53.65333681562498
45.52069932285802
43.888757041390896
43.59055510007048
43.53267475957588
43.521704815418445
43.52000508865184
43.51988831837717
43.51986393533422
43.5197200126972
43.51947657724945
43.519174250581514
43.51884096977504
43.51849265536257
43.518137774936775
43.517780683713546
43.51742356964943
43.517067501847436
43.516712977815395
43.516360205055115
43.516009244984176
43.515660086244075
43.5153126819553
43.51496696860082
43.51462287555465
43.514280329850344
43.51393925852721
43.51359958974592
43.51326125328059
43.51292418069555
43.51258830536555
43.51225356241801
43.51191988863928
43.51158722236429
43.51125550336082
43.510924672712626
43.51059467270403
43.510265446707194
43.50993693907161
43.509609095016934
43.50928186052751
43.50895518224953
43.508629007389516
43.50830328361458
43.50797795895329
43.50765298169748
43.507328300304096
43.507003863296944
43.50667961916801
43.50635551627773
43.50603150275391
43.50570752638929
43.50538353453638
43.50505947400028
43.5047352909

13.486900713849275
13.480674755423369
13.47439867784081
13.468081659581893
13.461732790883094
13.455361104498438
13.448975589407604
13.442585184892394
13.43619875272583
13.42982502576919
13.423472532082476
13.417149494724725
13.410863708732261
13.404622398254686
13.398432058413926
13.392298288017768
13.38622562067877
13.380217363032017
13.37427544949242
13.368400323243101
13.3625908528292
13.356844292801059
13.35115629529741
13.345520977304663
13.339931045641322
13.334377978612585
13.328852259935783
13.323343657184127
13.317841533912706
13.312335182112639
13.306814159961926
13.30126861921435
13.29568960709728
13.290069329250134
13.284401362869865
13.278680812568357
13.27290440514675
13.26707052319046
13.261179180757
13.255231947206108
13.249231827251158
13.243183106532866
13.237091172467013
13.230962319902835
13.224803550393228
13.218622372789959
13.212426611592974
13.206224228144773
13.200023158475961
13.193831170453413
13.187655741896938
13.181503960523159
13.175382445925033
13.16929

11.328656056397381
11.326455806184432
11.324259340652297
11.322066638376048
11.319877678473095
11.317692440618963
11.315510905064535
11.313333052654054
11.311158864844463
11.308988323725192
11.306821412039652
11.304658113207724
11.302498411349301
11.300342291309398
11.298189738684782
11.296040739851232
11.293895281993846
11.291753353137306
11.289614942179153
11.287480038924487
11.285348634121902
11.283220719503113
11.281096287822866
11.278975332902457
11.276857849675222
11.274743834234538
11.272633283884748
11.270526197194634
11.26842257405453
11.266322415735813
11.264225724954867
11.262132505939537
11.260042764500449
11.257956508105595
11.25587374595981
11.253794489088557
11.251718750426129
11.249646544910274
11.247577889580196
11.245512803682672
11.243451308781992
11.241393428877785
11.239339190529442
11.237288622986428
11.235241758328343
11.233198631610376
11.231159281019423
11.229123748036697
11.227092077611742
11.225064318343208
11.22304052267244
11.22102074708391
11.2190050523188

11.083090227906277
10.750846968337523
10.942898791626583
11.558789288782354
10.822528983685311
10.698921716531
10.80167152715079
10.821117882724351
10.773913808050153
10.899483978190721
11.437980686238278
11.47501783496337
11.616992525418631
10.809875400669496
10.689622666704283
10.933069877692432
11.083874047187068
10.934208337980744
10.97139666918544
10.902429507278997
11.058207992375054
10.878066230118248
11.47635636414157
10.889750589617607
11.086998990060945
10.760878018512022
10.71485510739931
10.747398838892808
10.673860525316396
11.03274773421924
11.618300758893696
10.772332367634405
10.831664202965827
10.69940953336976
10.88099530247019
10.803967513012962
10.669827239750415
10.883368688491585
10.668747299823462
10.922462605040772
11.37957362508497
10.81935413395629
10.658355347580915
11.512659588247613
10.765355870690515
10.658087851273432
11.302259252549666
10.92065025827321
11.359783733542134
10.855812861669175
10.792642303274132
10.841021755781549
10.781436541559211
10.6622

In [307]:
x[0][0][1].get_arrW()

[-0.7474459627459888,
 0.6890667522412938,
 -0.32684219764302463,
 0.6376508246108795]

In [308]:
def create_file_ANN():
    file = open("text_ann.txt", "w")
#     print(len(x[1]))
#     print(x[1])

    for i in range(0,len(x[1])):
        for j in range(0,x[1][i]):
            file.write(str(i));
            file.write(',')
            file.write(str(j));
            file.write('=')
            file.write(str(x[0][i][j].get_arrW()))
            file.write('|')
            file.write(str(x[0][i][j].get_arrdW()))
            file.write('|')
            file.write(str(x[0][i][j].get_out()))
            file.write('|')
            file.write(str(x[0][i][j].get_error()))
            file.write('\n');

    file.close()
# file = open("newfile.txt", "r")
# print(file.read())
# file.close()

In [309]:
create_file_ANN()

In [310]:
prediction_ann = predMyMLP(testData, 'target', x, 3)
print(prediction_ann)

    target  prediction
0        1           1
1        0           0
2        2           2
3        1           1
4        1           1
5        0           0
6        1           1
7        2           2
8        1           2
9        1           1
10       2           2
11       0           0
12       0           0
13       0           0
14       0           0


In [311]:
print(confusion_matrix(prediction_ann,'target','prediction', class_list))

Predicted  0  1  2
Actual            
0          6  0  0
1          0  5  1
2          0  0  3


In [312]:
testing_accuracy(prediction_ann,'target','prediction')

0.93

## CrosVal

# DTL

In [313]:
kf = KFold(n_splits=10,shuffle=True)

In [314]:
accuracy_DTL_KF = []
for train_index, test_index in kf.split(iris_X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = iris_X.loc[train_index], iris_X.loc[test_index]
    y_train, y_test = iris_y.loc[train_index], iris_y.loc[test_index]
    trainData,testData =  mergeData(X_train, X_test, y_train, y_test)
    model = c45(trainData, "target",True)
    prediction_dtl = pred_c45(testData, model, "target")
    accuracy_DTL_KF.append(testing_accuracy(prediction_dtl,'target','prediction'))

In [315]:
accuracy_DTL_KF

[0.93, 0.93, 0.87, 0.93, 0.93, 1.0, 1.0, 0.93, 0.87, 1.0]

In [316]:
round(np.mean(accuracy_DTL_KF)*100,2)

93.9

# MLP

In [ ]:
accuracy_MLP_KF = []
for train_index, test_index in kf.split(iris_X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = iris_X.loc[train_index], iris_X.loc[test_index]
    y_train, y_test = iris_y.loc[train_index], iris_y.loc[test_index]
    trainData,testData =  mergeData(X_train, X_test, y_train, y_test)
    hidden_node = [4,4,2]
    # itu yang 200nya diganti 1000 biar akurasinya tinggi
    model = MLP(trainData, 'target', hidden_node,1500,0.1)
    prediction_ann = predMyMLP(testData, 'target', x, 3)
    accuracy_MLP_KF.append(testing_accuracy(prediction_ann,'target','prediction'))

54.26493353169074
48.0438839456355
45.77469348026853
44.9133901115176
44.56040770489032
44.3891995447357
44.27785649659056
44.176692367022156
44.06399141623098
43.936869097390236
43.80474154580888
43.67122157718326
43.52870555725416
43.366257171115805
43.173078026849275
42.93756872288347
42.64564827512848
42.27886607775979
41.8114196382782
41.20558311431412
40.42100576377378
39.45788318264331
38.313989764152076
36.96925442832466
35.45062656762488
33.830178246629316
32.204263392210954
30.669124795077384
29.296495999302575
28.121217107143128
27.144774048349642
26.347639372842345
25.701617990262505
25.178036571567937
24.751682562538022
24.401943988459657
24.11257199783235
23.870960987229964
23.667375579270676
23.494282416320697
23.345820151169438
23.217394720131978
23.10537505177289
23.00686516407178
22.91953299759252
22.84148095535966
22.77114697040704
22.707227868955446
22.648618987130106
22.594365644459295
22.54362339986266
22.49562524495637
22.449655263707697
22.405029999875445
22.361

13.634755970358304
13.627334922474882
13.619860537653711
13.612311142538504
13.604658791322034
13.59686615911472
13.588881270050972
13.580628012524866
13.571987680634631
13.562759578703705
13.55256603451602
13.540585083311973
13.524631698601183
13.49724710177307
13.436350889465391
13.400290896184188
13.401420308051609
13.401275498370284
13.39994728703774
13.398641368401067
13.397432591131775
13.396335525050342
13.395351056471279
13.394477853898431
13.393713928298537
13.393056943882476
13.392504312777097
13.392053237054823
13.391700730090612
13.391443627671421
13.391278594041106
13.391202125920291
13.391210556355274
13.391300059527724
13.391466657207312
13.391706227246857
13.392014514341914
13.392387143163907
13.392819633901059
13.393307420190446
13.393845869383462
13.394430305051763
13.395056031604193
13.3957183608469
13.396412640275335
13.397134282840598
13.397878797881342
13.39864182286038
13.399419155492614
13.400206785801394
13.40100092759648
13.401798048831695
13.402594900277379
1

12.880753179131435
12.878152891346785
12.875557309791105
12.872966402862122
12.8703801391221
12.867798487297492
12.86522141627839
12.862648895118214
12.860080893033158
12.857517379401662
12.854958323763837
12.852403695820893
12.849853465434471
12.847307602626065
12.844766077576253
12.842228860624026
12.839695922266102
12.837167233156102
12.834642764103785
12.832122486074294
12.829606370187346
12.827094387716333
12.824586510087588
12.82208270887941
12.819582955821295
12.81708722279297
12.814595481823574
12.812107705090693
12.809623864919473
12.807143933781708
12.804667884294908
12.802195689221323
12.799727321467056
12.797262754081105
12.794801960254421
12.792344913318935
12.78989158674657
12.787441954148385
12.784995989273565
12.782553666008416
12.780114958375487
12.777679840532606
12.775248286771863
12.772820271518656
12.770395769330886
12.767974754897832
12.765557203039297
12.763143088704652
12.760732386971894
12.758325073046722
12.755921122261626
12.753520510074916
12.751123212069889

11.93968544764724
11.937805495948188
11.935927624937689
11.93405184006005
11.9321781465882
11.93030654962334
11.928437054095072
11.926569664761322
11.924704386208363
11.922841222850916
11.920980178932373
11.91912125852482
11.917264465529579
11.91540980367716
11.913557276527882
11.911706887472166
11.909858639730938
11.908012536356187
11.906168580231451
11.90432677407245
11.902487120427608
11.900649621678818
11.898814280042046
11.896981097568185
11.895150076143693
11.893321217491508
11.891494523171867
11.889669994583198
11.887847632963043
11.886027439388982
11.88420941477966
11.882393559895783
11.880579875341173
11.878768361563809
11.876959018856954
11.875151847360343
11.873346847061203
11.871544017795534
11.869743359249336
11.867944870959706
11.866148552316197
11.864354402562096
11.862562420795589
11.860772605971226
11.858984956901057
11.857199472256163
11.855416150567896
11.853634990229288
11.851855989496393
11.850079146489682
11.848304459195571
11.846531925467653
11.844761543028259
11

16.71258998917357
16.683992726170125
16.655477480634694
16.62703925162108
16.598672428265168
16.570370916466405
16.542128294100074
16.513937990262786
16.485793482256156
16.457688502443375
16.42961724596166
16.401574569710466
16.37355617318041
16.34555875258196
16.31758012132485
16.289619292046762
16.26167651787331
16.23375329315256
16.205852316284666
16.1779774192372
16.150133469746407
16.122326252985804
16.09456233965249
16.06684894706271
16.0391937990949
16.011604989815055
15.984090854512454
15.95665985077946
15.92932045127596
15.902081048977635
15.87494987504173
15.847934928930432
15.821043920094453
15.794284220311736
15.767662825669555
15.741186327146034
15.714860888765948
15.688692232355931
15.66268562799306
15.636845889317241
15.611177372956375
15.585683981388755
15.560369168638564
15.535235948266852
15.510286903180122
15.4855241968341
15.460949585460611
15.436564430990805
15.41236971439
15.388366049157163
15.364553694776763
15.340932569942563
15.317502265401295
15.29426205629113

...

11.400252022893632
11.424120792774819
11.317920922728922
11.316903554925378
11.265384285192406
11.795731225646838
12.301225471467301
11.580697747167509
11.395004193820135
11.246215836779658
11.393467120948506
11.460687557776984
11.192686485504483
11.386938632120291
11.464064512019881
11.195692793446742
11.384286062637578
11.447768141568181
11.165047377975297
11.382267589955584
11.467949434876783
11.225176222034575
11.393037409326714
11.520866604810987
11.918514319912202
11.176316551286646
11.389068941105279
11.516798214355518
11.968363097008641
11.660821422704537
11.282321470350922
11.585951778456241
13.996739762880594
12.693299738521043
12.277480813311584
11.335041241326522
11.13171855167573
11.361195608704534
11.444028347115108
11.166039223849832
11.3904656775013
11.5562851586367
14.11246051131551
12.301000443289919
11.329884586691776
11.558108594011223
12.239772309562287
11.182240199059512
11.557483860245696
12.211537066726162
11.175285749666848
11.553745903706227
12.202431646443525

17.358668474706967
17.324568769181525
17.290567043879197
17.25667111427728
17.222888633178524
17.18922707342576
17.155693711396346
17.122295611203448
17.089039609552135
17.055932301219993
17.02298002515547
16.990188851207943
16.957564567524052
16.925112668661683
16.89283834448735
16.86074646993282
16.828841595693486
16.797127939954194
16.765609381227218
16.734289452383432
16.703171335950643
16.672257860744235
16.641551499883516
16.61105437023473
16.580768233307662
16.550694497618668
16.52083422251753
16.491188123461736
16.461756578707046
16.432539637369704
16.403537028803257
16.374748173220716
16.346172193482868
16.317807927963663
16.28965394439593
16.261708554593923
16.2339698299438
16.206435617549133
16.179103556916015
16.151971097061065
16.125035513925344
16.098293927978975
16.071743321902982
16.04538055823921
16.019202396903353
15.993205512462039
15.96738651108119
15.94174194706064
15.916268338877671
15.890962184670537
15.865819977102412
15.84083821755497
15.816013429610079
15.7913

13.767705984979438
13.765687662097198
13.763677915988834
13.761676672420357
13.759683858294448
13.757699401819842
13.755723232672095
13.753755282145708
13.751795483297256
13.749843771079753
13.747900082468053
13.745964356575147
13.744036534759863
13.742116560725403
13.740204380609418
13.738299943065392
13.736403199335639
13.734514103316027
13.732632611612829
13.730758683591661
13.728892281419002
13.727033370096407
13.725181917487795
13.72333789434006
13.721501274297385
13.71967203390941
13.717850152633682
13.716035612832895
13.714228399766737
13.712428501579287
13.71063590928181
13.708850616731507
13.707072620606496
13.705301920377256
13.70353851827498
13.701782419257118
13.700033630970143
13.698292163710292
13.696558030382102
13.694831246455337
13.693111829920369
13.691399801242335
13.689695183314315
13.68799800140972
13.686308283134116
13.684626058376779
13.682951359261958
13.68128422010038
13.679624677340628
13.677972769521348
13.676328537223563
13.674692023023916
13.673063271448706

13.484571912129821
13.48297303989657
13.481364231140871
13.479745665064966
13.478117531574627
13.476480031496905
13.474833376800511
13.473177790818962
13.47151350847629
13.469840776515834
13.468159853731727
13.46647101120353
13.464774532533864
13.463070714089081
13.461359865243095
13.459642308624398
13.457918380366099
13.456188430359111
13.454452822508605
13.45271193499301
13.4509661605264
13.449215906622939
13.447461595864208
13.445703666168004
13.443942571058859
13.442178779938946
13.440412778358766
13.438645068286103
13.436876168371413
13.435106614207765
13.433336958582036
13.431567771713771
13.429799641477127
13.428033173599635
13.42626899183015
13.424507738066549
13.422750072431118
13.420996673278353
13.419248237116738
13.417505478421004
13.41576912930621
13.414039939027774
13.41231867326374
13.410606113125088
13.40890305382778
13.407210302945169
13.405528678141525
13.403859004265716
13.402202109657637
13.400558821489588
13.398929959927443
13.397316330853936
13.39571871684598
13.3

44.607266975304896
44.56270197817361
44.52906544358696
44.49862578287668
44.468681519717144
44.4381702748469
44.40653913516301
44.37337687056885
44.338332708846494
44.301178850694924
44.26195870355101
44.22112066979162
44.17946306039105
44.1377995475008
44.09654441338914
44.05554451799781
44.014219150257986
43.971799004705844
43.92748499949461
43.88049680367755
43.83005438139419
43.77533397361921
43.71541820981989
43.64924437747979
43.57554776403539
43.492794682080564
43.39909907936137
43.29211580567743
43.16890191506937
43.025734465782676
42.85786883341075
42.65921522833688
42.421902791839784
42.13569174882827
41.78719215940918
41.35887923108422
40.82802578395053
40.16599029743515
39.33877475014842
38.31039174737431
37.05231771549708
35.564032497302925
33.90059740270026
32.18110755745275
30.553150588735615
29.13270566339041
27.967649489592283
27.04589768840865
26.326200308667108
25.76242579643669
25.314261234508777
24.950089798551453
24.64680580529708
24.38848363892038
24.164515298808

In [ ]:
accuracy_MLP_KF

In [ ]:
round(np.mean(accuracy_MLP_KF)*100,2)